In [ ]:
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf

# Set plot styles
plt.rcParams['figure.dpi'] = 150

# Model function definition needed for fitting
# Identical to the one in your main notebook
def peak_with_erf_tail(E, A_peak, E_break, alpha1, alpha2, C_tail, E_trans, W_trans):
    if E_break <= 0:
        return np.full_like(E, np.inf)
    # Using numpy functions directly to avoid warnings in this context if desired
    term1 = (E / E_break)**(-alpha1)
    term2 = (E / E_break)**(-alpha2)
    peak = A_peak * (term1 + term2)**(-1)
    
    switch = (1 + erf((E - E_trans) / W_trans)) / 2.0
    
    return peak * (1 - switch) + C_tail * switch

In [ ]:
# Configuration
# Update this path to point to the cache directory created by the main notebook
# This is likely inside 'NewRMFsADP' in your working directory
CACHE_DIR = "/Users/leodrake/Documents/IXPE/NewRMFsADP/fit-cache-du123" 

def load_cached_results(cache_folder):
    if not os.path.exists(cache_folder):
        raise FileNotFoundError(f"Cache directory not found: {cache_folder}")

    # Find all peak files
    peak_files = glob.glob(os.path.join(cache_folder, "*sim-*-peak.npz"))
    
    energies = []
    peak_results = []
    low_results = []

    print(f"Found {len(peak_files)} cached files. Loading...")

    for p_file in peak_files:
        # Extract energy from filename (assuming format sim-XXXXX-...)
        match = re.search(r'sim-(\d{5})', os.path.basename(p_file))
        if match:
            # Convert 01000 -> 1.0 keV
            e_val = int(match.group(1)) / 1000.0
            
            # Construct corresponding 'low' filename
            l_file = p_file.replace("-peak.npz", "-low.npz")
            
            if os.path.exists(l_file):
                try:
                    # Load peak data
                    with np.load(p_file, allow_pickle=True) as d:
                        p_data = {k: d[k].item() if d[k].ndim == 0 else d[k] for k in d}
                    
                    # Load low data
                    with np.load(l_file, allow_pickle=True) as d:
                        l_data = {k: d[k].item() if d[k].ndim == 0 else d[k] for k in d}

                    energies.append(e_val)
                    peak_results.append(p_data)
                    low_results.append(l_data)
                except Exception as e:
                    print(f"Failed to load {e_val} keV: {e}")

    # Sort everything by energy
    if energies:
        sorted_indices = np.argsort(energies)
        return (np.array(energies)[sorted_indices], 
                [peak_results[i] for i in sorted_indices], 
                [low_results[i] for i in sorted_indices])
    else:
        return np.array([]), [], []

# Execute loader
all_energies_for_du, all_peak_results_for_du, all_low_results_for_du = load_cached_results(CACHE_DIR)
print(f"Successfully loaded {len(all_energies_for_du)} energy points.")

In [ ]:
# Configuration
alpha_bin_index = 6

# Custom Fit Conditions
# Parameters: [A_peak, E_break, alpha1, alpha2, C_tail, E_trans, W_trans]
# Indices:    [   0  ,    1   ,    2  ,    3  ,    4   ,     5   ,     6   ]

# Peak PI Data Setup
p0_peak_custom = [1.4, 3, 6, -2.0, 0.113, 6.0, 8]
bounds_peak_custom = ([0, 1.5, 5, -np.inf, 0, 0, 0.1], 
                      [2, 6, np.inf, 0, 1, np.inf, np.inf])
params_to_freeze_peak = [0, 1]

# Low PI Data Setup
p0_low_custom = [0.5, 4.0, 3.0, -2.0, 0.1, 7.0, 2.0]
bounds_low_custom = ([0, 1.0, 0, -np.inf, 0, 4, 0.1],
                     [1, 8, 5, 0, 1, 12, 5.0])
params_to_freeze_low = [] 


# Data Filtering
valid_indices = [
    i for i, (p, l, e) in enumerate(zip(all_peak_results_for_du, all_low_results_for_du, all_energies_for_du))
    if p is not None and l is not None and np.isfinite(e)
]
energies = np.array([all_energies_for_du[i] for i in valid_indices])
peak_results = [all_peak_results_for_du[i] for i in valid_indices]
low_results  = [all_low_results_for_du[i] for i in valid_indices]

# Extract Data for Chosen Alpha Bin
alpha_bin_centers = peak_results[0]["alpha_bins"]
d_alpha = 1.0 / len(alpha_bin_centers)
alpha_center = alpha_bin_centers[alpha_bin_index]
alpha_low, alpha_high = alpha_center - d_alpha/2.0, alpha_center + d_alpha/2.0
print(f"--- Fitting Alpha Bin #{alpha_bin_index} ({alpha_low:.2f} <= alpha < {alpha_high:.2f}) ---")

mu_peak_slice = np.array([res['mu_bins'][alpha_bin_index] for res in peak_results])
mu_err_peak_slice = np.array([res['mu_bins_err'][alpha_bin_index] for res in peak_results])
mu_low_slice = np.array([res['mu_bins'][alpha_bin_index] for res in low_results])
mu_err_low_slice = np.array([res['mu_bins_err'][alpha_bin_index] for res in low_results])

# Perform and Plot Fits
fig, ax = plt.subplots(figsize=(10, 7))
e_fit_plot = np.linspace(energies.min(), energies.max(), 200)
param_names = ['A_peak', 'E_break', 'alpha1', 'alpha2', 'C_tail', 'E_trans', 'W_trans']

def calculate_and_print_results(title, y_data, x_data, err_data, popt, frozen_indices=[]):
    residuals = y_data - peak_with_erf_tail(x_data, *popt)
    chi2 = np.sum((residuals / err_data)**2)
    dof = len(x_data) - (len(popt) - len(frozen_indices)) 
    reduced_chi2 = chi2 / dof if dof > 0 else np.inf
    print(f"\n--- {title} ---")
    print(f"   - Reduced Chi-Squared: {reduced_chi2:.3f} (Chi^2: {chi2:.2f}, DoF: {dof})")
    for i, (name, val) in enumerate(zip(param_names, popt)):
        status = "(frozen)" if i in frozen_indices else ""
        print(f"   - {name:<10}: {val:.3f} {status}")

# Peak PI Fits
mask_peak = np.isfinite(mu_peak_slice) & (mu_err_peak_slice > 0)
if np.sum(mask_peak) > 6:
    x_peak, y_peak, yerr_peak = energies[mask_peak], mu_peak_slice[mask_peak], mu_err_peak_slice[mask_peak]
    
    # 1. Original Fit
    try:
        p0_peak_orig = [1, 2, 6, -2.0, 0.1, 6.0, 2.0]
        bounds_peak_orig = ([0, 1.5, 5, -np.inf, 0, 4, 0.1], [2, 6, np.inf, 0, 1, 12, np.inf])
        popt_peak_orig, _ = curve_fit(peak_with_erf_tail, x_peak, y_peak, p0=p0_peak_orig, sigma=yerr_peak, maxfev=10000, bounds=bounds_peak_orig)
        ax.plot(e_fit_plot, peak_with_erf_tail(e_fit_plot, *popt_peak_orig), color='k', ls='-', lw=2, label='Original Fit (Peak PI)')
        calculate_and_print_results("Original Peak PI Fit Results", y_peak, x_peak, yerr_peak, popt_peak_orig)
    except (RuntimeError, ValueError) as e:
        print(f"\n--- Original Peak PI Fit FAILED: {e} ---")

    # 2. Custom Fit (with frozen parameters)
    try:
        free_indices = [i for i in range(len(p0_peak_custom)) if i not in params_to_freeze_peak]
        
        p0_free = [p0_peak_custom[i] for i in free_indices]
        bounds_free = ([bounds_peak_custom[0][i] for i in free_indices], 
                       [bounds_peak_custom[1][i] for i in free_indices])
        
        def wrapped_model(x, *p_free):
            p_full = list(p0_peak_custom)
            for i, idx in enumerate(free_indices):
                p_full[idx] = p_free[i]
            return peak_with_erf_tail(x, *p_full)
        
        popt_free, _ = curve_fit(wrapped_model, x_peak, y_peak, p0=p0_free, sigma=yerr_peak, maxfev=10000, bounds=bounds_free)
        
        popt_peak_custom = list(p0_peak_custom)
        for i, idx in enumerate(free_indices):
            popt_peak_custom[idx] = popt_free[i]
            
        ax.plot(e_fit_plot, peak_with_erf_tail(e_fit_plot, *popt_peak_custom), color='blue', ls=':', lw=2.5, label='Custom Fit (Peak PI)')
        calculate_and_print_results("Custom Peak PI Fit Results", y_peak, x_peak, yerr_peak, popt_peak_custom, frozen_indices=params_to_freeze_peak)
    except (RuntimeError, ValueError) as e:
        print(f"\n--- Custom Peak PI Fit FAILED: {e} ---")

# Plot formatting
ax.errorbar(energies, mu_peak_slice, yerr=mu_err_peak_slice, c='deeppink', fmt='.', capsize=3, label="Peak PI Data")
ax.set(
    title=f"Fit Comparison for Alpha Bin #{alpha_bin_index} ({alpha_low:.2f} ≤ α < {alpha_high:.2f})",
    xlabel="Energy (keV)",
    ylabel=r"$\mu$"
)
ax.legend(fontsize='small')
ax.grid(True, linestyle='--', alpha=0.6)
plt.show()